In [1]:
#Reference Code: https://github.com/aimacode/aima-python/blob/master/search.py

from queue import Queue
import copy
import time 
import tracemalloc

In [2]:
class Problem:
    
    def __init__(self, initial, goal=None):            #This constructor specifies the initial state
        self.initial = initial
        
    def domain_values(self, numbers, used):         #it returns the domain values
        return [number for number in numbers if number not in used]
    
    def find_empty(self, puzzle, state):            #Returns the empty position i.e, position with '0'
        for row in range(puzzle):
            for column in range(puzzle):
                if state[row][column] == 0:
                    return row, column
                
    def result(self, state, action):                   # Returns updated state after adding new valid value
        play = action[0]
        row = action[1]
        column = action[2]

        new_state = copy.deepcopy(state)         #This adds new valid value to the state, without repeating previous ones
        new_state[row][column] = play

        return new_state
    

    def goal_test(self, state):                #Tests the state and return true if reached to goal

        total = 45    #The sum of each row, colum, and small grid (quadrant)

        # Check rows and columns and return false if total is invalid
        for row in range(9):
            if (len(state[row]) != 9) or (sum(state[row]) != total):
                return False

            column_total = 0
            for column in range(9):
                column_total += state[column][row]

            if (column_total != total):
                return False

        # Check quadrants and return false if total is invalid
        for column in range(0,9,3):
            for row in range(0,9,3):

                grid_total = 0
                for grid_row in range(0,3):
                    for grid_column in range(0,3):
                        grid_total += state[row + grid_row][column + grid_column]

                if (grid_total != total):
                    return False
        return True

    def constraints(self, state):
        number_set = range(1, 10) #the set of values whixh can be put on board
        in_column = [] #the list of vaid values for column
        in_block = [] #the list of vaid values for block or quadrant

        row,column = self.find_empty(9, state) #get the first empty position 

        # Filter valid values based on row
        in_row = [number for number in state[row] if (number != 0)]
        options = self.domain_values(number_set, in_row)

        # Filter valid values based on column
        for column_index in range(9):
            if state[column_index][column] != 0:
                in_column.append(state[column_index][column])
        options = self.domain_values(options, in_column)

        # Filter with valid values based on block or quadrant
        row_start = int(row/3)*3
        column_start = int(column/3)*3
        
        for block_row in range(0, 3):
            for block_column in range(0,3):
                in_block.append(state[row_start + block_row][column_start + block_column])
        options = self.domain_values(options, in_block)

        for number in options:
            yield number, row, column

In [3]:
class Node:

    def __init__(self, state, action=None):   #This constructor creates a search tree with state and action 
        self.state = state
        self.action = action

    def expand(self, problem):   #Expand the problem and return list of nodes reachable in one step from this node
        return [self.child_node(problem, action) for action in problem.constraints(self.state)]

    def child_node(self, problem, action):   #return a node with new state
        next = problem.result(self.state, action)
        return Node(next, action)

In [6]:
def BFS(problem, node):
    steps=0
    if problem.goal_test(node.state):    #check if already at goal state
        steps+=1
        steps=1
        return node, steps

    temp_solution = Queue()         #FIFO queue for BFS
    temp_solution.put(node)       

    while (temp_solution.qsize() != 0):   # Loop until all nodes are explored or solution found
        a=0
        node = temp_solution.get()
        for child_node in node.expand(problem):
            steps+=1
            a+=1
            maxSteps=a
            for row in child_node.state:
                print (row)
            print('\n')
            if problem.goal_test(child_node.state):
                return child_node, steps

            temp_solution.put(child_node)

    return None

In [9]:
avgSteps=0
maxSteps=0
no=1
avgMemory=0
maxMemory=0
avgTime=0
maxTime=0
file = open("testData.txt","r")
                #recording the start time
for line in file:
    
    line=line.strip()
    example=line.replace('.','0')
    s=[]
    for i in example:
        s.append(int(i))
    puzzle=[[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]
           ,[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
    index=0
    for row in range(9):
        for column in range(9):
            puzzle[row][column] = s[index]
            index+=1
    for row in puzzle:
        print(row)
    print('\n')
    tree = Problem(puzzle)
    root_node = Node(tree.initial)
    start = time.time() 
    tracemalloc.start()
    solution, steps = BFS(tree, root_node)
    current, peak = tracemalloc.get_traced_memory()
    elapsedTime = time.time() - start
    avgTime +=elapsedTime
    avgMemory +=peak
    avgSteps +=steps

    if(peak>maxMemory):
        maxMemory = peak
    
    
    if(steps>maxSteps):
        maxSteps = steps
        
    if(elapsedTime>maxTime):
        maxTime=elapsedTime
    
    if solution:
        print ("Found solution: ",no)
        no+=1
           # for row in solution.state:
             #   print (row)
    else:
        print ("No possible solutions")

file.close()
print(f"Total steps:", avgSteps)
print(f"Average steps:", avgSteps/50)
print(f"Maximun steps:", maxSteps)
print(f"Average time:", avgTime/50)
print(f"Maximum Time:", maxTime)
print(f"Average Memory Used: {(avgMemory/50)/ 10**6}MB")
print(f"Maximum Memory Used: {(maxMemory)/ 10**6}MB")

[0, 0, 3, 0, 2, 0, 6, 0, 0]
[9, 0, 0, 3, 0, 5, 0, 0, 1]
[0, 0, 1, 8, 0, 6, 4, 0, 0]
[0, 0, 8, 1, 0, 2, 9, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 0, 3, 0, 2, 0, 6, 0, 0]
[9, 0, 0, 3, 0, 5, 0, 0, 1]
[0, 0, 1, 8, 0, 6, 4, 0, 0]
[0, 0, 8, 1, 0, 2, 9, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 0, 3, 0, 2, 0, 6, 0, 0]
[9, 0, 0, 3, 0, 5, 0, 0, 1]
[0, 0, 1, 8, 0, 6, 4, 0, 0]
[0, 0, 8, 1, 0, 2, 9, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 5, 3, 0, 2, 0, 6, 0, 0]
[9, 0, 0, 3, 0, 5, 0, 0, 1]
[0, 0, 1, 8, 0, 6, 4, 0, 0]
[0, 0, 8, 1, 0, 2, 9, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1

[4, 8, 3, 9, 2, 1, 6, 7, 5]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[5, 2, 1, 8, 7, 6, 4, 3, 0]
[0, 0, 8, 1, 0, 2, 9, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 7, 5]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[5, 2, 1, 8, 7, 6, 4, 9, 0]
[0, 0, 8, 1, 0, 2, 9, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 0]
[0, 0, 8, 1, 0, 2, 9, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 5, 0]
[0, 0, 8, 1, 0, 2, 9, 0, 0]
[7, 0, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1

[9, 6, 7, 3, 4, 5, 8, 2, 1]
[5, 2, 1, 8, 7, 6, 4, 9, 3]
[3, 4, 8, 1, 5, 2, 9, 7, 6]
[7, 1, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[5, 2, 1, 8, 7, 6, 4, 9, 3]
[3, 4, 8, 1, 5, 2, 9, 7, 6]
[7, 5, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[5, 2, 1, 8, 7, 6, 4, 9, 3]
[3, 4, 8, 1, 5, 2, 9, 7, 6]
[7, 9, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[5, 2, 1, 8, 7, 6, 4, 9, 3]
[3, 4, 8, 1, 6, 2, 9, 7, 5]
[7, 1, 0, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9,

[7, 1, 9, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 5]
[4, 3, 8, 1, 5, 2, 9, 7, 6]
[7, 2, 9, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 5]
[4, 3, 8, 1, 5, 2, 9, 7, 6]
[7, 5, 9, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 5]
[4, 5, 8, 1, 3, 2, 9, 7, 6]
[7, 1, 9, 0, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 5]
[4, 5, 8, 1,

[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 5, 3]
[4, 5, 8, 1, 3, 2, 9, 7, 6]
[7, 2, 9, 5, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 5, 3]
[4, 5, 8, 1, 3, 2, 9, 7, 6]
[7, 3, 9, 5, 0, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[2, 5, 1, 8, 7, 6, 4, 9, 3]
[3, 4, 8, 1, 5, 2, 9, 7, 6]
[7, 1, 9, 4, 3, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[2, 5, 1, 8, 7, 6, 4, 9, 3]
[3, 4, 8, 1, 5, 2, 9, 7, 6]
[7, 1, 9, 4, 6, 0, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1

[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 5]
[3, 5, 8, 1, 6, 2, 9, 7, 4]
[7, 1, 9, 5, 3, 4, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 5]
[3, 5, 8, 1, 6, 2, 9, 7, 4]
[7, 2, 9, 5, 3, 4, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 5]
[4, 5, 8, 1, 3, 2, 9, 7, 6]
[7, 1, 9, 5, 6, 4, 0, 0, 8]
[0, 0, 6, 7, 0, 8, 2, 0, 0]
[0, 0, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[5, 8, 3, 4, 2, 1, 6, 9, 7]
[9, 6, 4, 3, 7, 5, 8, 2, 1]
[2, 7, 1, 8, 9, 6, 4, 3, 5]
[4, 5, 8, 1, 3, 2, 9, 7, 6]
[7, 2, 9, 5, 6, 4, 0, 0, 8]
[0, 0, 6, 7,

[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[2, 5, 1, 8, 7, 6, 4, 9, 3]
[5, 4, 8, 1, 3, 2, 9, 7, 6]
[7, 2, 9, 5, 6, 4, 1, 3, 8]
[1, 3, 6, 7, 9, 8, 2, 4, 5]
[3, 7, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[2, 5, 1, 8, 7, 6, 4, 9, 3]
[5, 4, 8, 1, 3, 2, 9, 7, 6]
[7, 2, 9, 5, 6, 4, 1, 3, 8]
[3, 1, 6, 7, 9, 8, 2, 4, 5]
[1, 3, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 5, 7]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[2, 5, 1, 8, 7, 6, 4, 9, 3]
[5, 4, 8, 1, 3, 2, 9, 7, 6]
[7, 2, 9, 5, 6, 4, 1, 3, 8]
[3, 1, 6, 7, 9, 8, 2, 4, 5]
[1, 7, 2, 6, 0, 9, 5, 0, 0]
[8, 0, 0, 2, 0, 3, 0, 0, 9]
[0, 0, 5, 0, 1, 0, 3, 0, 0]


[4, 8, 3, 9, 2, 1, 6, 7, 5]
[9, 6, 7, 3, 4, 5, 8, 2, 1]
[2, 5, 1, 8, 7, 6, 4, 9, 3]
[3, 4, 8, 1, 6, 2, 9, 5, 7]
[7, 2, 9, 5, 3, 4, 1, 6, 8]
[5, 1, 6, 7, 9, 8, 2, 3, 4]
[1, 3, 2, 6,